In [1]:
!pip install konlpy

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import numpy as np


In [3]:
data = pd.read_csv('./플레저_result3.csv', encoding='cp949')
data.head()

,Names,level,writeDate,score,tag,Reviews,keywords,similar,label
0,가쥭가바앙Lv.1,Lv.1,2024.12.10 (수정됨),10,#얼굴지방이식,납작한 이마랑 얼굴 볼륨감 라인감 플레저에서 얼굴지방이식으로 해결한지 한달! 지방 ...,"""한 번의 시술로 평생 유지"",",0.501629,1
1,립밤밤밤밤Lv.1,Lv.1,2024.12.09 (수정됨),10,#눈밑지방재배치,나이먹으면서 생긴 눈밑 지방.. 툭튀어나오니 주변사람들이 어디 아프냐고 하는데 거울...,"""한 번의 시술로 평생 유지"",",0.551615,1
2,popmaru6Lv.1,Lv.1,2025.03.18 (수정됨),10,#얼굴지방이식,매우 유익하고 입소문이 좋아서 골랐습니다! 도착하고 나서 대기 시간도 없이 바로 상...,"""지금 신청하세요!"",",0.631025,1
3,unniWmp8pwLv.2,Lv.2,2025.03.19 (수정됨),9,#눈밑지방재배치,플레저 성형외과 정재은 원장님께 부탁드렸습니다. 예산이 맞았던 것과 증례가 깔끔했기...,"""한 번의 시술로 평생 유지"",",0.683709,1
4,unniCACukGLv.1,Lv.1,2025.03.19 (수정됨),10,#이마지방이식 #PRP자가지방이식,저는 이마가 평평한 것이 콤플렉스여서 병원을 찾고 있었는데 싸고 증례가 좋은 병원이...,"""리프팅 효과 반영구적"",",0.489824,1


In [4]:
data['label'].value_counts()

1    5347
0     925
Name: label, dtype: int64

In [5]:
# 한글 텍스트 전처리
import re
from konlpy.tag import Okt

okt = Okt()

# 텍스트 전처리 함수
def clean_text(text):
    text = re.sub(r'[^가-힣\s]', '', text)  # 한글, 공백만 남김
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# 형태소 분석 함수
def tokenize(text):
    return okt.morphs(text, stem=True)  # 어간 추출까지

data['Reviews'] = data['Reviews'].apply(lambda x: clean_text(str(x)))
print(data['Reviews'].head())

data['Reviews'] = data['Reviews'].apply(lambda x: tokenize(str(x)))
print(data['Reviews'].head())



0    납작한 이마랑 얼굴 볼륨감 라인감 플레저에서 얼굴지방이식으로 해결한지 한달 지방 빠...
1    나이먹으면서 생긴 눈밑 지방 툭튀어나오니 주변사람들이 어디 아프냐고 하는데 거울보니...
2    매우 유익하고 입소문이 좋아서 골랐습니다 도착하고 나서 대기 시간도 없이 바로 상담...
3    플레저 성형외과 정재은 원장님께 부탁드렸습니다 예산이 맞았던 것과 증례가 깔끔했기 ...
4    저는 이마가 평평한 것이 콤플렉스여서 병원을 찾고 있었는데 싸고 증례가 좋은 병원이...
Name: Reviews, dtype: object
0    [납작하다, 이마, 랑, 얼굴, 볼륨, 감, 라인, 감, 플, 레저, 에서, 얼굴,...
1    [나이, 먹다, 생기다, 눈밑, 지방, 툭, 튀어나오다, 주변, 사람, 들, 이, ...
2    [매우, 유익하다, 입, 소문, 이, 좋다, 고르다, 도착, 하고, 나서다, 대기,...
3    [플, 레저, 성형, 외과, 정재은, 원장, 님, 께, 부탁드리다, 예산, 이, 맞...
4    [저, 는, 이마, 가, 평평하다, 것, 이, 콤플렉스, 여서, 병원, 을, 찾다,...
Name: Reviews, dtype: object


In [6]:
data['Reviews'] = data['Reviews'].apply(lambda tokens: ' '.join(tokens))
data['Reviews'][0]

'납작하다 이마 랑 얼굴 볼륨 감 라인 감 플 레저 에서 얼굴 지 방이 식 으로 해결 한지 한 달 지방 빠지다 울퉁불퉁하다 곳 없이 매끈하다 자다 채우다 너무 만족하다 있다 생 지방 쓰기 도 하다 원장 님 께서 딱 필요하다 곳 들 꼼꼼하다 수술 해주다 그렇다 같다 예전 에는 사진 찍다 외출 하다 때 괜히 얼굴 라인 가리다 보정 하고 그랬는데 이제 는 완전 당당 이대로 유지 오 랬 동안 해주다 생각 이고 앞 으로 얼굴 라인 관리 볼륨 관리 는 플 레저 여기 서 받다 생각 이에요'

In [7]:
# 정상 리뷰만 담은 데이터프레임
df_norm = data[data['label'] == 0]
print(len(df_norm['Reviews']))

# 허위인 리뷰만 담은 데이터프레임
df_fake = data[data['label'] == 1]
print(len(df_fake['Reviews']))

925
5347


In [8]:
# TF-IDF 벡터라이징

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(tokenizer=tokenize, max_features=5000)
X = vectorizer.fit_transform(data['Reviews'])
y = data['label']

In [9]:
# 데이터 분리 작업

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# # 랜덤포레스트 사용
# # 구린거 알지만 그냥 써봄

# from sklearn.ensemble import RandomForestClassifier
# model = RandomForestClassifier()
# model.fit(X_train, y_train)

# y_pred = model.predict(X_test)
# print(classification_report(y_test, y_pred))

# # 사용결과 : 역시나 구림 ㅋㅋ

In [11]:
# # 나이브베이즈 사용
# # 특징 : 텍스트 특화, 빠름

# from sklearn.naive_bayes import MultinomialNB

# model = MultinomialNB()
# model.fit(X_train, y_train)

# 사용결과 : 판별 정확도가 너무 떨어짐

In [12]:
# SVM 사용
# 특징 : 높은 정확도

# from sklearn.svm import LinearSVC

# model = LinearSVC()
# model.fit(X_train, y_train)


# 가중치 수정본
from sklearn.svm import LinearSVC

model = LinearSVC(class_weight='balanced')
model.fit(X_train, y_train)

LinearSVC(class_weight='balanced')

In [13]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.52      0.71      0.60       180
           1       0.95      0.89      0.92      1075

    accuracy                           0.86      1255
   macro avg       0.73      0.80      0.76      1255
weighted avg       0.89      0.86      0.87      1255



In [14]:
def predict_review(text):
    cleaned = clean_text(text)
    vectorized = vectorizer.transform([cleaned])
    pred = model.predict(vectorized)[0]
    return "허위 리뷰" if pred == 1 else "정상 리뷰"

# 사용 예시
print(predict_review("플레처 브이컷주사 내돈내산후기 결제는 2세트로 6회 100만원이였고 1세트할때보다 가격 저렴해쥠ㅇㅇ 부가세 따로있고 예약비는 2만 5층에서 접수하고 이것저것 설문조사 같은거 하는데 질문이 디게 꼼꼼해서 먼가 좋아씀 나에대해 잘 알아보고 시술해주는 느낌? 실장님상담은 친절했고 원장님 상담 간단하게 한뒤 시술 받았어여 어차피 쁘띠 시술이라 성형수술처럼 길게 물어볼건 없었고 성분이랑 효과나오는 시기정도만 ㅇㅇ 지금 3회차까지 맞았고 중간 점검할겸 후기 남겨봄 일단 주사는 개인차있겠지만 1도 안아팠고 대신 당일에 식빵맨처럼 부었음...ㅋㅋ 효과는 나름 보이는 중인거 같고 턱살쪽 빠지니까 얼굴도 좀 선명해보이는거같은.. 이목구비 또렷하게 보이는느낌^^ 6회차까지 다 맞으면 여기서 더빠지겠죠?? 다맞고 한번 더 남겨볼게요"))

허위 리뷰


In [15]:
def predict_review(text):
    cleaned = clean_text(text)
    vectorized = vectorizer.transform([cleaned])
    pred = model.predict(vectorized)[0]
    return "허위 리뷰" if pred == 1 else "정상 리뷰"

# 사용 예시
print(predict_review('지방분해주사 고민하시는 분들 플레처 추천해요! 👍👍 이번 여름에 결혼식을 올릴 예정이라 급하게 다이어트가 필요한 상황이었습니다.. 특히 배는 튼살이 생길만큼 갑자기 살이 쪘었는데요 ㅠㅠ 컷주사 덕분에 찐살 도로 안드로메다로 돌려보냈습니다 ^^ 사실 이렇게 얄상하게 빠지는 것 까진 기대를 안했는데 3회차 이후부터는 쭉쭉 빠지더라고요 ㅎㅎ 1회 맞았을 때 변화가 크지 않아서 조바심 냈던 거 생각하면 부끄부끄하구 그래요..ㅠ_ㅠ 시간이 답이고 투자해서 관리하는 건 배신하지 않더라구용~! ㅎㅎ 앞으로도 식단관리 잘해서 유지 해보려구 해용 ㅎㅎ 아 그리구 처음 상담 잡을 때 상담비 2만원 있는데 시술 가격에서 빼서 적용해주십니당!! 참고하시라구 적어봐요~'))

허위 리뷰


In [16]:
print(predict_review(input(str)))

허위 리뷰


In [17]:
data_orig = pd.read_csv('./플레저_result3.csv', encoding='cp949')
data_orig.head()

,Names,level,writeDate,score,tag,Reviews,keywords,similar,label
0,가쥭가바앙Lv.1,Lv.1,2024.12.10 (수정됨),10,#얼굴지방이식,납작한 이마랑 얼굴 볼륨감 라인감 플레저에서 얼굴지방이식으로 해결한지 한달! 지방 ...,"""한 번의 시술로 평생 유지"",",0.501629,1
1,립밤밤밤밤Lv.1,Lv.1,2024.12.09 (수정됨),10,#눈밑지방재배치,나이먹으면서 생긴 눈밑 지방.. 툭튀어나오니 주변사람들이 어디 아프냐고 하는데 거울...,"""한 번의 시술로 평생 유지"",",0.551615,1
2,popmaru6Lv.1,Lv.1,2025.03.18 (수정됨),10,#얼굴지방이식,매우 유익하고 입소문이 좋아서 골랐습니다! 도착하고 나서 대기 시간도 없이 바로 상...,"""지금 신청하세요!"",",0.631025,1
3,unniWmp8pwLv.2,Lv.2,2025.03.19 (수정됨),9,#눈밑지방재배치,플레저 성형외과 정재은 원장님께 부탁드렸습니다. 예산이 맞았던 것과 증례가 깔끔했기...,"""한 번의 시술로 평생 유지"",",0.683709,1
4,unniCACukGLv.1,Lv.1,2025.03.19 (수정됨),10,#이마지방이식 #PRP자가지방이식,저는 이마가 평평한 것이 콤플렉스여서 병원을 찾고 있었는데 싸고 증례가 좋은 병원이...,"""리프팅 효과 반영구적"",",0.489824,1


In [18]:
# 실제 적용
data_orig['conclusion'] = data_orig['Reviews'].apply(lambda x: predict_review(str(x)))

In [19]:
data_orig.head

data_orig['conclusion'].value_counts()

허위 리뷰    4917
정상 리뷰    1355
Name: conclusion, dtype: int64

In [20]:
data_orig.to_csv('플레저_결과물_가중치.csv', index=False, encoding='utf-8-sig')

In [21]:
import os
os.getcwd()

'c:\\아나콘다\\프로젝트'

In [23]:
# 가중치 안넣은 결과물
# pd.read_csv('./플레저_결과물.csv', encoding='utf-8')['conclusion'].value_counts()

In [ ]:
# 가중치 넣은 결과물 (살짝 과적합)
pd.read_csv('./플레저_결과물_가중치.csv', encoding='utf-8')['conclusion'].value_counts()

허위 리뷰    4917
정상 리뷰    1355
Name: conclusion, dtype: int64